In [9]:
import pandas as pd
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import cv2

In [14]:
data = pd.read_csv('solution.csv')
data.head()

,id,category
0,1,5
1,2,4
2,3,6
3,4,2
4,5,2


In [17]:
directory = 'training/training'
for filename in os.listdir(directory):
    old_file = os.path.join(directory, filename)
    new = str(data['id'][int(filename.split('.')[0])-1]) + '_' + str(data['category'][int(filename.split('.')[0])-1]) + '.png'
    new_file = os.path.join(directory, new)
    os.rename(old_file, new_file)

KeyError: 16913

In [10]:
import keras
import wide_residial_network as wrn

ModuleNotFoundError: No module named 'wide_residial_network'

In [7]:
!git clone https://github.com/EricAlcaide/keras-wrn

Cloning into 'keras-wrn'...
remote: Enumerating objects: 107, done.
remote: Total 107 (delta 0), reused 0 (delta 0), pack-reused 107
Receiving objects: 100% (107/107), 96.72 KiB | 77.00 KiB/s, done.
Resolving deltas: 100% (45/45), done.


In [18]:
import keras
import keras.backend as K 
from keras.models import Model
from keras.layers import Dense, Dropout, Add, Input, BatchNormalization, Activation
from keras.layers import  Conv2D, MaxPooling2D, AveragePooling2D, Flatten
from keras.regularizers import l2

def main_block(x, filters, n, strides, dropout):
	# Normal part
	x_res = Conv2D(filters, (3,3), strides=strides, padding="same")(x)# , kernel_regularizer=l2(5e-4)
	x_res = BatchNormalization()(x_res)
	x_res = Activation('relu')(x_res)
	x_res = Conv2D(filters, (3,3), padding="same")(x_res)
	# Alternative branch
	x = Conv2D(filters, (1,1), strides=strides)(x)
	# Merge Branches
	x = Add()([x_res, x])

	for i in range(n-1):
		# Residual conection
		x_res = BatchNormalization()(x)
		x_res = Activation('relu')(x_res)
		x_res = Conv2D(filters, (3,3), padding="same")(x_res)
		# Apply dropout if given
		if dropout: x_res = Dropout(dropout)(x)
		# Second part
		x_res = BatchNormalization()(x_res)
		x_res = Activation('relu')(x_res)
		x_res = Conv2D(filters, (3,3), padding="same")(x_res)
		# Merge branches
		x = Add()([x, x_res])

	# Inter block part
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	return x

def build_model(input_dims, output_dim, n, k, act= "relu", dropout=None):
	""" Builds the model. Params:
			- n: number of layers. WRNs are of the form WRN-N-K
				 It must satisfy that (N-4)%6 = 0
			- k: Widening factor. WRNs are of the form WRN-N-K
				 It must satisfy that K%2 = 0
			- input_dims: input dimensions for the model
			- output_dim: output dimensions for the model
			- dropout: dropout rate - default=0 (not recomended >0.3)
			- act: activation function - default=relu. Build your custom
				   one with keras.backend (ex: swish, e-swish)
	"""
	# Ensure n & k are correct
	assert (n-4)%6 == 0
	assert k%2 == 0
	n = (n-4)//6 
	# This returns a tensor input to the model
	inputs = Input(shape=(input_dims))

	# Head of the model
	x = Conv2D(16, (3,3), padding="same")(inputs)
	x = BatchNormalization()(x)
	x = Activation('relu')(x)

	# 3 Blocks (normal-residual)
	x = main_block(x, 16*k, n, (1,1), dropout) # 0
	x = main_block(x, 32*k, n, (2,2), dropout) # 1
	x = main_block(x, 64*k, n, (2,2), dropout) # 2
			
	# Final part of the model
	x = AveragePooling2D((8,8))(x)
	x = Flatten()(x)
	outputs = Dense(output_dim, activation="softmax")(x)

	model = Model(inputs=inputs, outputs=outputs)
	return model


if __name__ == "__main__":
	model = build_model((32,32,3), 10, 22, 8)
	model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 32, 32, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_39 (BatchNo (None, 32, 32, 16)   64          conv2d_45[0][0]                  
__________________________________________________________________________________________________
activation_39 (Activation)      (None, 32, 32, 16)   0           batch_normalization_39[0][0]     
__________________________________________________________________________________________________
conv2d_46 

In [19]:
path = 'solution.csv'
data = pd.read_csv(path)
data.head()

,id,category
0,1,5
1,2,4
2,3,6
3,4,2
4,5,2


In [20]:
train_data = []
labels = []
train_path = 'training/training'
for file in tqdm(os.listdir(train_path)):
    path = os.path.join(train_path, file)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (100, 100))
    train_data.append(np.array(img))
    labels.append(data['category'][int(file.split('.')[0])-1])

  0%|          | 0/5000 [00:00<?, ?it/s]


KeyError: 16913

In [22]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [23]:
X = train_data
X = np.array(X).reshape(len(X),100,100,1)
X = X/255.0
y = np.array(to_categorical(labels))

ValueError: zero-size array to reduction operation maximum which has no identity

In [10]:
shape, classes = (32, 32, 3), 6

model = build_model(shape, classes, 16, 4)

model.compile("adam", "categorical_crossentropy", ["accuracy"])

(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

y_train = keras.utils.np_utils.to_categorical(y_train)

model.fit(x_train, y_train, epochs=10)

  3612672/170498071 [..............................] - ETA: 10:55

KeyboardInterrupt: 